In [1]:
import os
import pickle

import piton.extension.dataloader
import msgpack
import numpy as np

data_path = "/local-scratch/nigam/projects/ethanid/piton_1_extract"

dictionary_path = (
    "/local-scratch/nigam/projects/ethanid/piton/native/results/dictionary"
)

surv_dictionary_path = (
    "/local-scratch/nigam/projects/ethanid/piton/native/results/survival_clmbr_dictionary"
)

import piton.datasets

data = piton.datasets.PatientDatabase(data_path)
male_code = data.get_code_dictionary().index("Gender/M")

import json

dictionary = msgpack.load(open(dictionary_path, 'rb'), use_list=False)

if True:
    surv_dict = msgpack.load(open(surv_dictionary_path, 'rb'), use_list=False)
    print(surv_dict.keys())
    task = {"type": "survival_clmbr", "survival_dict": surv_dict, "dim": 256}
elif False:
    task = {"type": "clmbr", 'vocab_size': 10_000}
else:
    labels = []

    if False:
        limit = 100
    else:
        limit = len(data)

    for patient_id in range(0, limit):
        patient = data[patient_id]
        is_male = any(event.code == male_code for event in patient.events)
        labels.append((patient.patient_id, 1, is_male))
    task = {"type": "binary", "labels": labels}

config = {
    "transformer": {
        "vocab_size": 50000,
        "dictionary": dictionary,
        "max_size": 13,
        "hidden_size": 768,
        "intermediate_size": 3072,
        "n_heads": 12,
        "n_layers": 6,
    
        "rotary": "disabled",
    },

    "task": task,
    "seed": 97,
    "splits": [["train", 0, 70], ["dev", 70, 85], ["test", 85, 100]],
    "learning_rate": 1e-3,
    "max_grad_norm": 1.0,
    "l2": 0,

    "n_epochs": 100,
}

print('WORKING WITH', config['learning_rate'])


with open("trash/config.json", "bw") as f:
    msgpack.dump(config, f)

loader = piton.extension.dataloader.BatchCreator(data_path, "trash/config.json")

print("Starting to load ...")

dict_keys(['codes', 'time_bins'])
WORKING WITH 0.001
Starting to load ...


In [2]:
for i in range(100):

    b1 = loader.get_batch("train", i)

    # print(len(b1['patient_ids']))

    if len(b1['patient_ids']) > 100:
        break
    
print(i, b1)
p = data[b1['patient_ids'][0]]


17 {'patient_ids': array([19480, 13677, 19559,  9983, 20283, 22835, 13054, 12294,  9344,
       18922, 19277,  5803, 10297,  6944, 25692,  9758, 18833, 26966,
       18251, 12444,  1712, 10159, 12019, 24685,  4828, 27334, 22648,
        6877, 23575, 15141, 19891, 11093,  2284, 14407, 20592,  1355,
        3053, 16710,  8716, 23986, 10112, 11340, 14732, 15811, 17968,
       25818,  8239, 14772,  3197, 18004,  5606, 21135,  9110,  2419,
       21533,  6919, 22568, 21335, 25116, 22099, 13776, 10701,  3482,
        3530, 11977, 12567, 16563, 23813,  1666,  3368, 11676, 17244,
       27345,  8014, 19540, 16137, 23563, 22300, 25742,  6299, 24073,
       20027, 14196, 20211, 22878, 18962, 26344, 16319,  3841,  6920,
       23918,  5677, 21234,  6579, 10273, 19271, 17491,  3868,  6050,
        5350, 24995, 23826, 11343, 11637, 21710, 11201, 10039, 16855,
        8980,  2551, 21509, 15039, 21872, 16440,  7027, 23439, 25514,
       27266,  2188,  6492, 21470, 20314, 10941, 14595,  9178,   569,
 

In [24]:
d = data.get_code_dictionary()

def h(c):
    return bytes(d[c]).decode('utf8')

for event in p.events:
    print(event.start, (event.start - p.events[0].start).days/365, h(event.code), type(event.value))

1951-01-28 00:00:00 0.0 SNOMED/3950001 <class 'NoneType'>
1951-01-28 23:59:00 0.0 Gender/M <class 'NoneType'>
1951-01-28 23:59:00 0.0 Ethnicity/Not Hispanic <class 'NoneType'>
2018-01-23 16:55:00 67.03287671232877 Visit/OP <class 'NoneType'>
2018-03-02 02:06:00 67.13698630136986 LOINC/28570-0 <class 'memoryview'>
2018-03-02 23:59:00 67.13698630136986 Domain/OMOP generated <class 'NoneType'>
2018-03-11 10:00:00 67.16164383561645 LOINC/LP173418-7 <class 'memoryview'>
2018-03-11 10:00:00 67.16164383561645 LOINC/11506-3 <class 'memoryview'>
2018-03-11 10:04:00 67.16164383561645 LOINC/8480-6 <class 'float'>
2018-03-11 10:04:00 67.16164383561645 LOINC/8277-6 <class 'float'>
2018-03-11 10:04:00 67.16164383561645 LOINC/8462-4 <class 'float'>
2018-03-11 10:04:00 67.16164383561645 LOINC/8310-5 <class 'float'>
2018-03-11 10:04:00 67.16164383561645 LOINC/8310-5 <class 'float'>
2018-03-11 10:04:00 67.16164383561645 LOINC/9279-1 <class 'float'>
2018-03-11 10:04:00 67.16164383561645 LOINC/8327-9 <cla

In [4]:
tran = b1['transformer']
print(tran['ages'][0, :] / 365)
print(tran['label_indices'])

print(config['task']['survival_dict']['time_bins'])

[0.0000000e+00 2.7378234e-03 2.7378234e-03 6.7034805e+01 6.7137230e+01
 6.7139725e+01 6.7162796e+01 6.7162796e+01 6.7162796e+01 6.7162796e+01
 6.7162796e+01 6.7162796e+01 6.7162796e+01 6.7162796e+01 6.7162796e+01
 6.7162796e+01 6.7162796e+01 6.7162796e+01 6.7162796e+01 6.7164383e+01
 6.7164383e+01 6.7164383e+01 6.7164383e+01 6.7164383e+01 6.7164383e+01
 6.7164383e+01 6.7164383e+01 6.7164383e+01 6.7164383e+01 6.7164383e+01
 6.7164383e+01 6.7164383e+01 6.7164383e+01 6.7164383e+01 6.7164383e+01
 6.7164383e+01 6.7164383e+01 6.7164383e+01 6.7165596e+01 6.7167122e+01
 6.7167122e+01 6.7167122e+01 6.7168983e+01 6.7172600e+01 6.7180824e+01
 6.7180824e+01 7.1561646e+01 7.1561646e+01 7.1568153e+01 7.1630135e+01
 7.1630135e+01 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00]
[   3    4   18 ... 8192 8192 8192]
(0.0, 0.427734375, 0.5849609375, 1.455078125, 4.423828

In [28]:
task = b1['task']
print(2 ** (task['sparse_time'][1][:16]))


print('time bins', config['task']['survival_dict']['time_bins'][8])
print('time bins', [h(a) for a in config['task']['survival_dict']['codes'][:10]])
print('age', tran['ages'][0, 3] / 365)

print(task['event_indices'][:10,:])
print(task['sparse_time'][0][:16])
print(task['sparse_time'][2][:50])
print(2**(task['sparse_time'][3][:16]))

[4.27734375e-01 1.57226562e-01 8.70117188e-01 2.96875000e+00
 4.15917969e+00 6.78320312e+00 8.91894627e+00 1.17148438e+01
 1.86445332e+01 3.00097637e+01 4.58720703e+01 7.64736252e+01
 1.24332016e+02 2.12667984e+02 4.96611755e+02 0.00000000e+00]
time bins 36.0
time bins ['Domain/OMOP generated', 'SNOMED/138875005', 'Visit/OP', 'CPT4/1002795', 'Visit/IP', 'LOINC/LP29693-6', 'LOINC/PANEL', 'LOINC/LP7787-7', 'LOINC/CHEM', 'LOINC/LG55-6']
age 67.03480843321918
[[ 8  0]
 [ 8  1]
 [ 8  2]
 [ 8  3]
 [ 8  4]
 [ 8 11]
 [ 8 20]
 [ 8 21]
 [ 8 23]
 [ 8 29]]
[ 0  0  0  0  0  0  0  0  0 87 87 87 87 87 87 87]
[   0    1    2    3    4   11   20   21   23   29   33   54   64   74
   82   92   97  121  134  138  178  187  209  238  244  327  333  369
  400  476  489  541  573  589  610  636  682  711  824  829  895  911
  912  919  963  977 1001 1002 1011 1016]
[ 2.2949219 11.294923  11.294923  11.294923  11.294923  11.294923
 11.294923  11.294923  11.294923  11.294923  11.294923  11.294923
 11.294923  